In [52]:
import pandas as pd

path = r"C:\\Users\\valef\\Downloads\\kaggle (1).csv"

df = pd.read_csv(path, encoding='latin-1')

print(df)

         ï»¿Lat         Lng What Dinosaurs Eat              Accepted Name  \
0     42.933300  123.966698              PLANT      Chaoyangsaurus youngi   
1     41.799999  120.733330   PLANT and ANIMAL  Protarchaeopteryx robusta   
2     41.799999  120.733330   PLANT and ANIMAL           Caudipteryx zoui   
3     50.740726 -111.528732              FLESH       Gorgosaurus libratus   
4     50.737015 -111.549347              FLESH       Gorgosaurus libratus   
...         ...         ...                ...                        ...   
2458  49.183334  -98.099998               FISH          Hesperornis chowi   
2459  49.183334  -98.099998               FISH     Hesperornis macdonaldi   
2460  49.183334  -98.099998               FISH     Hesperornis macdonaldi   
2461  49.183334  -98.099998               FISH          Hesperornis chowi   
2462  49.183334  -98.099998               FISH     Hesperornis macdonaldi   

     Country  Cc       Diet   Early Interval      Formation  \
0      China

In [53]:
#Because I my target is 'what Dinosaurs eat', I am only interested in keeping the columns which are related with it.

In [54]:
df.drop(['ï»¿Lat', 'Lng', 'Accepted Name', 'Cc', 'Early Interval', 'Early Interval', 'Ref Author', 'Ref Pubyr'], axis=1, inplace=True)



In [55]:
print(df)

     What Dinosaurs Eat Country       Diet      Formation Geological Interval  \
0                 PLANT   China  herbivore      Tuchengzi           Tithonian   
1      PLANT and ANIMAL   China   omnivore         Yixian           Barremian   
2      PLANT and ANIMAL   China   omnivore         Yixian           Barremian   
3                 FLESH  Canada  carnivore  Dinosaur Park           Campanian   
4                 FLESH  Canada  carnivore  Dinosaur Park           Campanian   
...                 ...     ...        ...            ...                 ...   
2458               FISH  Canada  piscivore   Pierre Shale           Campanian   
2459               FISH  Canada  piscivore   Pierre Shale           Campanian   
2460               FISH  Canada  piscivore   Pierre Shale           Campanian   
2461               FISH  Canada  piscivore   Pierre Shale           Campanian   
2462               FISH  Canada  piscivore   Pierre Shale           Campanian   

     Geological Time Period

In [56]:
df

,What Dinosaurs Eat,Country,Diet,Formation,Geological Interval,Geological Time Period,State,Max Ma,Min Ma
0,PLANT,China,herbivore,Tuchengzi,Tithonian,Jurassic,Liaoning,150.8,132.90
1,PLANT and ANIMAL,China,omnivore,Yixian,Barremian,Cretaceous,Liaoning,130.0,122.46
2,PLANT and ANIMAL,China,omnivore,Yixian,Barremian,Cretaceous,Liaoning,130.0,122.46
3,FLESH,Canada,carnivore,Dinosaur Park,Campanian,Cretaceous,Alberta,83.5,70.60
4,FLESH,Canada,carnivore,Dinosaur Park,Campanian,Cretaceous,Alberta,83.5,70.60
...,...,...,...,...,...,...,...,...,...
2458,FISH,Canada,piscivore,Pierre Shale,Campanian,Cretaceous,Manitoba,83.5,70.60
2459,FISH,Canada,piscivore,Pierre Shale,Campanian,Cretaceous,Manitoba,83.5,70.60
2460,FISH,Canada,piscivore,Pierre Shale,Campanian,Cretaceous,Manitoba,83.5,70.60
2461,FISH,Canada,piscivore,Pierre Shale,Campanian,Cretaceous,Manitoba,83.5,70.60


In [57]:
# Print the count of missing values in each column, in order to visualize if it is necessary to fill any of them.
print(df.isnull().sum())

What Dinosaurs Eat          0
Country                     0
Diet                        0
Formation                 111
Geological Interval         0
Geological Time Period      0
State                       0
Max Ma                      0
Min Ma                      0
dtype: int64


In [58]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Convert categorical data to one-hot encoded format
# One-hot encoding is used to convert categorical data into a format that can be used by machine learning algorithms.
df = pd.get_dummies(df, columns=["Country", "Diet", "Geological Time Period", "State", 'Formation', 'Geological Interval'])

# Separating features and target variable
# We separate the input features (X) from the target variable (y).
X = df.drop('What Dinosaurs Eat', axis=1)
y = df['What Dinosaurs Eat']

# Label encoding the target variable
# Convert the categorical target variable into numerical labels.
le = LabelEncoder()
y = le.fit_transform(y)

# Splitting data into training and test set
# This ensures that we have a separate dataset to train our model and another dataset to test its performance.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
# Scaling the features ensures that they have similar scales. This is crucial for algorithms like SVM which are sensitive to feature scales.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train SVM classifiers with different kernels
# SVM (Support Vector Machine) is a supervised machine learning algorithm which can be used for both classification or regression problems.
# The kernel transforms the data into a higher dimension to make it linearly separable. Here we're using the polynomial kernel.
kernels = ['poly']
accuracies = {}

for kernel in kernels:
    if kernel == 'poly':
        # For the polynomial kernel, we're experimenting with different values of C and degree.
        # C is the regularization parameter. A smaller value of C creates a wider margin, which may allow some misclassifications.
        # A larger value of C creates a narrower margin, aiming to classify all training examples correctly.
        # Degree specifies the degree of the polynomial kernel function. A higher degree can capture more complex relationships but can also overfit.
        for c in [10]:
            for degree in [2, 3, 4]:
                clf = SVC(kernel=kernel, C=c, degree=degree)
                clf.fit(X_train, y_train)  # Training the SVM classifier
                y_pred = clf.predict(X_test)  # Predicting using the trained classifier
                accuracies[f"{kernel}_C_{c}_degree_{degree}"] = accuracy_score(y_test, y_pred)  # Storing accuracy of the model

print(accuracies) 

# Print the accuracies for each configuration
#The SVM with the polynomial kernel is chosen to capture the non-linear relationships in the data.
#By varying the degree of the polynomial and the regularization parameter (C), the code tests different
#configurations of the SVM classifier to find the optimal settings for the problem. 
#The results (accuracies) will provide insights into which combination of hyperparameters work best for this dataset.

{'poly_C_10_degree_2': 0.7647058823529411, 'poly_C_10_degree_3': 0.845841784989858, 'poly_C_10_degree_4': 0.7281947261663286}
